In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import tensorflow as tf

#from keras.utils import plot_model
import pydot
import pydotplus
from pydotplus import graphviz
from keras.utils.vis_utils import plot_model
from keras.utils.vis_utils import model_to_dot
keras.utils.vis_utils.pydot = pydot
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse, r2_score as r2
import tensorflow_datasets as tfds
from sklearn.datasets import load_boston

#from keras.models import Sequential
#from keras.layers import Dense
#from keras.utils import to_categorical
#from keras.datasets import mnist

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split

### 1. Постройте нейронную сеть(берем простую линейную сеть, которую разбирали на уроке: меняем число слоев, число нейронов , типы активации, тип оптимизатора) на датасет from sklearn.datasets import load_boston.

In [3]:
boston = load_boston()

boston_df = pd.DataFrame(boston.data, columns=boston.feature_names)
boston_df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


In [4]:
train_data = boston.data
train_target = boston.target


In [5]:
train_data, test_data, train_target, test_target = train_test_split(train_data, train_target, test_size=0.20)


In [43]:
model_boston = Sequential([
    Dense(64, activation='relu', input_shape=(13,)),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

model_boston.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                     loss='mse',
                     metrics=['mae'])   #mae

model_boston.summary()

Model: "sequential_2488"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7464 (Dense)           (None, 64)                896       
_________________________________________________________________
dense_7465 (Dense)           (None, 32)                2080      
_________________________________________________________________
dense_7466 (Dense)           (None, 1)                 33        
Total params: 3,009
Trainable params: 3,009
Non-trainable params: 0
_________________________________________________________________


In [49]:
model_boston.fit(
    train_data,
    train_target,
    epochs=5,
    verbose = 0
)

In [50]:
model_boston.evaluate(train_data, train_target)


13/13 [==============================] - 0s 1ms/step - loss: 82.5021 - mae: 6.8124


[82.50210571289062, 6.812440872192383]

In [51]:
model_boston.evaluate(test_data, test_target)


4/4 [==============================] - 0s 3ms/step - loss: 75.4781 - mae: 6.7454


[75.47811126708984, 6.745428562164307]

In [52]:
predictions = model_boston.predict(test_data)


In [53]:
r2(test_target, predictions)


0.08894578269639652

### 2. Измените функцию потерь и метрику для этой задачи. Постройте 10-15 вариантов и сведите результаты их работы в таблицу Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?

In [56]:
results = []
for opt in ['adam', 'RMSprop', 'Adamax']:
  for n in [2, 8, 32, 64]:
    for metrics in ['mean_absolute_error', 'mean_squared_error']:
      for loss in ['mse', 'mae']:
        for epoch in [1, 3, 5, 7]:
          model_boston = Sequential([
          Dense(n, activation='relu', input_shape=(13,)),
          Dense(n/2, activation='relu'),
          Dense(1, activation='linear')
          ])

          i = 0

          result = {}

          model_boston.compile(optimizer=opt,
                        loss=loss,
                        metrics=metrics)
            
          model_boston.fit(train_data, train_target, epoch,
                        verbose = False)
          predictions = model_boston.predict(test_data)

          result['n'] = n
          result['opt'] = opt
          result['metrics'] = metrics
          result['loss'] = loss
          result['epoch'] = epoch
          i += 1
          result['r2'] = round(r2(test_target, predictions.copy().flatten()),5)
          result['rmse'] = round(np.sqrt(mse(test_target, predictions.copy().flatten())),5)
          result['mse'] = round(mse(test_target, predictions.copy().flatten()),5)
          result['mae'] = round(mae(test_target, predictions.copy().flatten()),5)


          results.append(result)

result = pd.DataFrame(results)
result

,n,opt,metrics,loss,epoch,r2,rmse,mse,mae
0,2,adam,mean_absolute_error,mse,1,-23.75023,45.28225,2050.48187,44.17309
1,2,adam,mean_absolute_error,mse,3,-495.96891,202.90979,41172.38089,127.19989
2,2,adam,mean_absolute_error,mse,5,-5.64822,23.46882,550.78540,21.63188
3,2,adam,mean_absolute_error,mse,7,-7.25018,26.14388,683.50256,24.91698
4,2,adam,mean_absolute_error,mae,1,-5.48074,23.17131,536.90969,21.30875
...,...,...,...,...,...,...,...,...,...
187,64,Adamax,mean_squared_error,mse,7,-0.39056,10.73330,115.20364,7.73615
188,64,Adamax,mean_squared_error,mae,1,-0.00059,9.10471,82.89578,7.18945
189,64,Adamax,mean_squared_error,mae,3,0.28858,7.67717,58.93888,6.29492
190,64,Adamax,mean_squared_error,mae,5,0.20443,8.11856,65.91099,6.17303


In [57]:
result.sort_values(by='r2', ascending=False).head(10)


,n,opt,metrics,loss,epoch,r2,rmse,mse,mae
169,32,Adamax,mean_squared_error,mse,3,0.46127,6.68070,44.63178,5.41003
48,64,adam,mean_absolute_error,mse,1,0.43456,6.84432,46.84475,5.21849
181,64,Adamax,mean_absolute_error,mae,3,0.42979,6.87316,47.24028,4.83865
164,32,Adamax,mean_absolute_error,mae,1,0.40743,7.00663,49.09289,5.29675
109,32,RMSprop,mean_squared_error,mae,3,0.39774,7.06364,49.89504,5.12122
34,32,adam,mean_absolute_error,mse,5,0.39593,7.07428,50.04538,5.39301
121,64,RMSprop,mean_squared_error,mse,3,0.37255,7.20987,51.98225,5.34835
32,32,adam,mean_absolute_error,mse,1,0.36083,7.27690,52.95325,5.54915
108,32,RMSprop,mean_squared_error,mae,1,0.35967,7.28348,53.04903,5.42718
101,32,RMSprop,mean_absolute_error,mae,3,0.35853,7.29000,53.14407,5.48522


Вывод: результат значительно вырос, но у изначальной модели он был ужасен... Лучшие результаты показали модели с 32 и 64 нейронами при 3 и 1 эпохе. Из оптимизаторов лучше всех показал себя Adamax